In [1]:
from langchain_tavily import TavilySearch
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Initialize TavilySearch with the API key from environment variables
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")


In [20]:
tool = TavilySearch(
    max_results=2,
    topic="general",
    include_raw_content = True
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)
res = tool.invoke({"query": "Explain the first LLAMA paper in detail. Only use arxiv"})

In [21]:
res

{'query': 'Explain the first LLAMA paper in detail. Only use arxiv',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Title: LLaMA: Open and Efficient Foundation Language Models - arXiv.org',
   'url': 'https://arxiv.org/abs/2302.13971',
   'content': 'Title:LLaMA: Open and Efficient Foundation Language Models View a PDF of the paper titled LLaMA: Open and Efficient Foundation Language Models, by Hugo Touvron and 13 other authors (or arXiv:2302.13971v1 [cs.CL] for this version) View a PDF of the paper titled LLaMA: Open and Efficient Foundation Language Models, by Hugo Touvron and 13 other authors - [x] Bibliographic Explorer Toggle  - [x] Connected Papers Toggle  - [x] Litmaps Toggle  - [x] scite.ai Toggle  - [x] alphaXiv Toggle  - [x] Links to Code Toggle  - [x] DagsHub Toggle  - [x] GotitPub Toggle  - [x] Huggingface Toggle  - [x] Links to Code Toggle  - [x] ScienceCast Toggle  - [x] Replicate Toggle  - [x] Spaces Toggle  - [x] Spaces Toggle  - [

In [22]:
print(res['results'][0]['raw_content'])

[2302.13971] LLaMA: Open and Efficient Foundation Language Models

[![Image 1: close this message](https://arxiv.org/static/browse/0.3.4/images/icons/close-slider.png)](https://arxiv.org/abs/2302.13971#)

![Image 2: arXiv smileybones](https://arxiv.org/static/browse/0.3.4/images/icons/smileybones-pixel.png)

arXiv Is Hiring a DevOps Engineer
---------------------------------

Work on one of the world's most important websites and make an impact on open science.

[**View Jobs**](https://info.arxiv.org/hiring/index.html)

[Skip to main content](https://arxiv.org/abs/2302.13971#content)

[![Image 3: Cornell University Logo](https://arxiv.org/static/browse/0.3.4/images/icons/cu/cornell-reduced-white-SMALL.svg)](https://www.cornell.edu/)

arXiv Is Hiring a DevOps Engineer

[View Jobs](https://info.arxiv.org/hiring/index.html)

We gratefully acknowledge support from the Simons Foundation, [member institutions](https://info.arxiv.org/about/ourmembers.html), and all contributors.[Donate](https